In [1]:
import gc
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import random

from os import path
from keras.utils import image_dataset_from_directory
from matplotlib import pyplot as plt
from keras import layers, losses, optimizers, metrics, callbacks

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9048)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

0 Physical GPU, 0 Logical GPUs


In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 32

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
train_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'train.csv')).drop(["Unnamed: 0"], axis=1)
dataset_train = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'train'),
    labels=list(train_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_train = dataset_train.map(lambda x, y : (x, tf.one_hot(y, N_CLASS)))

val_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'val.csv')).drop(["Unnamed: 0"], axis=1)
dataset_val = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'val'),
    labels=list(val_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_val = dataset_val.map(lambda x, y: (x, tf.one_hot(y, N_CLASS)))

test_labels_df = pd.read_csv(path.join(os.getcwd(), 'train_val_test', 'test.csv')).drop(["Unnamed: 0"], axis=1)

dataset_test = image_dataset_from_directory(
    directory=path.join(os.getcwd(), 'train_val_test', 'test'),
    labels=list(test_labels_df['codes']),
    label_mode='int',
    validation_split=0,
    shuffle=True,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512,
    seed=SEED
)
dataset_test = dataset_test.map(lambda x, y: (x, tf.one_hot(y, N_CLASS)))

Found 35000 files belonging to 1 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Found 5000 files belonging to 1 classes.
Found 10000 files belonging to 1 classes.


In [6]:
def create_model():
    model = keras.Sequential()

    model.add(keras.Input(shape=(32, 32, 3)))

    model.add(layers.Conv2D(32, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.Conv2D(32, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.Conv2D(64, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.Conv2D(64, 3, activation="relu", kernel_initializer='random_normal', bias_initializer='zeros'))
    model.add(layers.MaxPooling2D(2))

    # Fully connected classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dense(N_CLASS, activation="softmax"))
    return model

In [7]:
TRAINING_SEEDS = list(range(10))
results = []
opt = [
    optimizers.Adam(),
    optimizers.RMSprop(),
    optimizers.SGD()
]
early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

for o in opt:
    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        model = create_model()

        model.compile(
            optimizer=o,
            loss=losses.CategoricalCrossentropy(),
            metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
        )

        history = model.fit(
            dataset_train,
            epochs=200,
            validation_data=dataset_val,
            shuffle=True,
            callbacks=[early_stopping]
        )

        eval_results = model.evaluate(dataset_test)
        results += [{
            'seed': seed,
            'optimizer': o.name,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('optim_results.csv')

Epoch 1/200
69/69 [==============================] - 65s 919ms/step - loss: 3.0847 - categorical_accuracy: 0.2361 - categorical_crossentropy: 3.0847 - val_loss: 1.8496 - val_categorical_accuracy: 0.3128 - val_categorical_crossentropy: 1.8496
Epoch 2/200
 5/69 [=>............................] - ETA: 1:05 - loss: 1.8119 - categorical_accuracy: 0.3438 - categorical_crossentropy: 1.8119

KeyboardInterrupt: 

In [8]:
TRAINING_SEEDS = list(range(10))
results_lr = []
lr = {
    '0.001': 0.001,
    '0.0001': 0.0001,
    '0.00001': 0.00001,
    'default+scheduler': 0.001
}
early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

for lr_name, lr_val in lr.items():
    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        model = create_model()

        model.compile(
            optimizer=optimizers.Adam(learning_rate=lr_val),
            loss=losses.CategoricalCrossentropy(),
            metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
        )

        if lr_name == 'default+scheduler':
            reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, min_lr=0.00001, verbose=1)

            history = model.fit(
                dataset_train,
                epochs=200,
                validation_data=dataset_val,
                shuffle=True,
                callbacks=[early_stopping, reduce_lr]
            )
        else:
            history = model.fit(
                dataset_train,
                epochs=200,
                validation_data=dataset_val,
                shuffle=True,
                callbacks=[early_stopping]
            )

        eval_results = model.evaluate(dataset_test)
        results_lr += [{
            'seed': seed,
            'lr_strategy': lr_name,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        gc.collect()

results_lr = pd.DataFrame(results_lr)
results_lr = pd.concat([results_lr.drop(["results"], axis=1), results_lr["results"].apply(pd.Series)], axis=1)
results_lr.to_csv('learning_rate_results.csv')

Epoch 1/200
69/69 [==============================] - 75s 1s/step - loss: 3.1827 - categorical_accuracy: 0.1995 - categorical_crossentropy: 3.1827 - val_loss: 1.8900 - val_categorical_accuracy: 0.3000 - val_categorical_crossentropy: 1.8900
Epoch 2/200
19/69 [=======>......................] - ETA: 51s - loss: 1.7884 - categorical_accuracy: 0.3488 - categorical_crossentropy: 1.7884

KeyboardInterrupt: 